In [4]:
# Config Variables

# Define UCI Datasets to pull
# https://archive.ics.uci.edu/ml/datasets.php
uci_datasets = [165]

# Use all data (vs just training data) for OLS to determine co-efficient sign
ols_use_all_data = True

# Training set sizes to use (only training set sizes < 60% of total dataset size will be used)
#training_set_sizes = [10, 40, 160, 640, 2560, 10240, 40960]
training_set_sizes = [10, 40, 160, 610, 2560, 10240, 40960]
chain_lengths = [20000, 20000, 100000, 100000, 100000, 100000, 100000]

# Number of etas (prior standard deviation) to test between 1e-04 and 1e05 (recommend 20)
# Limits at 0 and infinity are calculated separately
# Note: In practice, sigma^2 (likelihood variance) is set to 1/eta^2 and
#       lambda (penalty) is set to 1/eta^4 
num_etas = 20

# Number of folds for cross-validation (set to 300 in 2018 paper, recommend testing with 5 to see if it works)
cv_folds = 20

# Overwrite model if it's already saved to file
overwrite_model = False

In [2]:
# Load necessary libraries and helper functions

# import half_ridge functions from helpers/half_ridge.py
from helpers.half_ridge import *

# import testing functions from helpers/testing_functions.py
from helpers.testing_functions import *

# import dataset class
from helpers.model import Model

# import os and pickle
import os
import pickle
 
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np
np.random.seed(42)

import time

In [5]:
# Load the ICU dataset, normalize the data, and split into training, validation, and test sets

# Loop over each dataset, loaded from file and formatted
for dataset in uci_datasets:
    model = Model()
    model.import_from_file('uci_ml_datasets/ucirepo_' + str(dataset) + '.csv')
    model.format_data()
    training_set_values = model.set_up_training_values(training_set_sizes, .6)

    # Check if a file exists for the dataset in the model_results folder
    if os.path.isfile(f"model_results/ucirepo_{dataset}.csv") and not overwrite_model:
        print(f"Model already exists for dataset {dataset}. Skipping...")
        continue

    # Print the training set values for this dataset
    print(f"Training Set Values: {training_set_values}")
    
    # Initialize the comparison results
    binary_comparisons = {}

    # loop over the training set values
    for training_set_index, training_set_value in enumerate(training_set_values):

        # Generate cross-validation indices
        cv = model.cv_indexing(cv_folds, training_set_value)

        # Initialize the comparison results for this training set value
        binary_comparisons[training_set_value] = {}

        # Vary eta values from 10^-4 to 10^5, including 0 and inf, and loop over them
        etas = np.logspace(-4, 5, num=num_etas)
        etas = np.insert(etas, 0, 0)
        etas = np.append(etas, np.inf)
        for eta in etas:

            # Initialize the comparison results for this eta
            binary_comparisons[training_set_value][eta] = []

            # Crossvalidate over the folds for a specific eta
            for i in range(cv_folds):

                # Print info about this iteration
                print(f"Training Set Value: {training_set_value}, Eta: {eta}, Fold: {i} of {range(cv_folds)}")

                # Get training and test indices for the i-th fold
                train_indices = np.where(cv[i] == 1)[0]
                test_indices = np.where(cv[i] == 0)[0]
                
                # Split the data
                X_train, X_test = model.X.iloc[train_indices], model.X.iloc[test_indices]
                y_train, y_test = model.y.iloc[train_indices], model.y.iloc[test_indices]

                # Determine the signs of the coefficients using OLS
                ols_coefficients = model.determine_signs_of_coefficients(ols_use_all_data, X_train, y_train)

                # Run the half ridge regression with mcmc
                #half_ridge_trace = half_ridge(X_train, y_train, model.ols_coefficients, prior_eta=eta)
                #var_dict = {}
                #for variable in half_ridge_trace.posterior:
                #    var_dict[variable] = half_ridge_trace.posterior[variable].values.flatten()
                # Results into a dataframe
                #var_weights = pd.DataFrame(var_dict)
                # Means for all the weights
                #var_means_array = var_weights.mean(axis=0)
                # Convert the dataframe var_means_array to a dictionary with the column names as keys
                #half_ridge_coefficients = {col: var_means_array[col] for col in var_weights.columns}
                
                # Run the half ridge regression with rejection sampling
                start_time = time.time()
                half_ridge_coefficients = half_ridge_rejection_sampling(ols_coefficients, X_train, y_train, eta, chain_lengths[training_set_index])
                end_time = time.time()
                duration = end_time - start_time
                print(f"Half ridge regression rejection sampling took {duration} seconds")

                # Calculate the binary comparisons for the models
                sorted_half_ridge_coefficients = {k: half_ridge_coefficients[k] for k in sorted(half_ridge_coefficients)}
                sorted_half_ridge_coefficients['intercept'] = 0
                print('half-ridge coefficients')
                print(sorted_half_ridge_coefficients)
                binary_comparison = calculate_binary_comparison(sorted_half_ridge_coefficients, X_test, y_test)
                binary_comparisons[training_set_value][eta].append(binary_comparison)
                print('binary comparisons')
                print(binary_comparison)   

    
    with open('model_results/ucirepo_' + str(dataset) + '.csv', 'wb') as file:
        pickle.dump(binary_comparisons, file)


Training Set Values: [10, 40, 160, 610]
Training Set Index: 0, Training Set Value: 10
Training Set Value: 10, Eta: 0.0, Fold: 0 of range(0, 20)
Running half-ridge regression...
{'Cement': 0.7978845608028654, 'Superplasticizer': 0.7978845608028654, 'Age': 0.7978845608028654, 'Water': 0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Fly Ash': 0.7978845608028654}
Half ridge regression rejection sampling took 8.511543273925781e-05 seconds
half-ridge coefficients
{'Age': 0.7978845608028654, 'Blast Furnace Slag': -0.7978845608028654, 'Cement': 0.7978845608028654, 'Coarse Aggregate': -0.7978845608028654, 'Fine Aggregate': -0.7978845608028654, 'Fly Ash': 0.7978845608028654, 'Superplasticizer': 0.7978845608028654, 'Water': 0.7978845608028654, 'intercept': 0}
binary comparisons
0.589
Training Set Value: 10, Eta: 0.0, Fold: 1 of range(0, 20)
Running half-ridge regression...
{'Cement': 0.7978845608028654

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.551
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 5 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.003662109375 seconds
half-ridge coefficients
{'Age': 0.7977481209015864, 'Blast Furnace Slag': -0.060200860793941144, 'Cement': 0.536123589819322, 'Coarse Aggregate': -0.13623613976125076, 'Fine Aggregate': -0.28068609324032523, 'Fly Ash': 0.1733215399726621, 'Superplasticizer': 0.39687239755466963, 'Water': 0.23890225656714792, 'intercept': 0}
binary comparisons
0.663
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 6 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.0038750171661376953 seconds
half-ridge coefficients
{'Age': 0.1298862336004335, 'Blast Furnace Slag': -0.10881932641577244, 'Cement': 0.8713638777035767, 'Coarse Aggregate': -0.4119471886539573, 'Fine Aggregate': -0.7475222513218251, 'Fly Ash': 0.2942427864908681, 'Superplasticizer': 0.51667744

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.579
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 11 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.0038492679595947266 seconds
half-ridge coefficients
{'Age': 0.6143872811935522, 'Blast Furnace Slag': -0.3763899498603223, 'Cement': 0.9588146481993852, 'Coarse Aggregate': -0.08380153551281748, 'Fine Aggregate': -0.17462397483183323, 'Fly Ash': 0.4658145980602885, 'Superplasticizer': 0.5738864961980078, 'Water': 0.06623670837490485, 'intercept': 0}
binary comparisons
0.619
Training Set Value: 10, Eta: 1.8329807108324339, Fold: 12 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.0038802623748779297 seconds
half-ridge coefficients
{'Age': 0.14121073505403114, 'Blast Furnace Slag': -0.32697171625994503, 'Cement': 0.32384479933305654, 'Coarse Aggregate': -0.14920302867211058, 'Fine Aggregate': -0.4830126236377438, 'Fly Ash': 0.1938821381179947, 'Superplasticizer': 

...

Running half-ridge regression...
Half ridge regression rejection sampling took 0.02083301544189453 seconds
half-ridge coefficients
{'Age': 99670315.87961653, 'Blast Furnace Slag': -105915528.75895822, 'Cement': 101242801.80171444, 'Coarse Aggregate': -106941048.35203476, 'Fine Aggregate': -100834373.45645043, 'Fly Ash': 99586036.06079832, 'Superplasticizer': 95160084.92371315, 'Water': 104426038.89747287, 'intercept': 0}
binary comparisons
0.593
Training Set Value: 610, Eta: 11288.378916846883, Fold: 3 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.02003192901611328 seconds
half-ridge coefficients
{'Age': 99355796.6390559, 'Blast Furnace Slag': -103511860.83564398, 'Cement': 106272855.92180158, 'Coarse Aggregate': -102448951.33534625, 'Fine Aggregate': -96017060.42124373, 'Fly Ash': 105736485.52316155, 'Superplasticizer': 101648801.26063581, 'Water': 96139820.5169828, 'intercept': 0}
binary comparisons
0.597
Training Set Value: 610, Eta

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)
/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.69
Training Set Value: 610, Eta: 33598.18286283774, Fold: 2 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.021793127059936523 seconds
half-ridge coefficients
{'Age': 821858111.4978701, 'Blast Furnace Slag': -963031622.0666147, 'Cement': 904523254.9458271, 'Coarse Aggregate': -852090481.4915257, 'Fine Aggregate': -920456723.2497097, 'Fly Ash': 885258683.348391, 'Superplasticizer': 941159750.5651735, 'Water': 946887465.4530506, 'intercept': 0}
binary comparisons
0.575
Training Set Value: 610, Eta: 33598.18286283774, Fold: 3 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.019981861114501953 seconds
half-ridge coefficients
{'Age': 909229131.9374267, 'Blast Furnace Slag': -902955435.1308624, 'Cement': 915836106.1033362, 'Coarse Aggregate': -930093716.8927119, 'Fine Aggregate': -887044406.7242098, 'Fly Ash': 875794340.0419734, 'Superplasticizer': 930350870.375839, 'Water': 

/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)
/Library/WebServer/Sites/SURF/bayesian_modeling/helpers/half_ridge.py:123: RuntimeWarning: Mean of empty slice
  post_weights = np.nanmean(samples1, axis=0)


binary comparisons
0.689
Training Set Value: 610, Eta: 100000.0, Fold: 2 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.020253896713256836 seconds
half-ridge coefficients
{'Age': 7318765609.099796, 'Blast Furnace Slag': -7704137338.234251, 'Cement': 8172809791.093563, 'Coarse Aggregate': -8450588611.157449, 'Fine Aggregate': -8533886669.123381, 'Fly Ash': 8163785772.930177, 'Superplasticizer': 8157467567.191031, 'Water': 8104477100.600599, 'intercept': 0}
binary comparisons
0.58
Training Set Value: 610, Eta: 100000.0, Fold: 3 of range(0, 20)
Running half-ridge regression...
Half ridge regression rejection sampling took 0.01925206184387207 seconds
half-ridge coefficients
{'Age': 7716792745.678491, 'Blast Furnace Slag': -7738271021.727921, 'Cement': 8146226897.444394, 'Coarse Aggregate': -8863476377.361597, 'Fine Aggregate': -7895163097.8874855, 'Fly Ash': 7754481019.537472, 'Superplasticizer': 8560484610.249069, 'Water': 7585692528.98868